### St up necessary bits

In [1]:
import sqlite3
import pandas as pd
from mpl_toolkits.mplot3d import axes3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
from datetime import datetime
from datetime import datetime, timedelta

## Bring in the Data from APSIM and prepare for graphing

In [2]:
def WetnessData(i):
    Wetness = []
    for l in range(1,LayerNum+1):
        Wetness.append(list(PoreWater.ix[i+1,'L'+str(l)].values))
    return np.array(Wetness)

In [5]:
con = sqlite3.connect('C:\ApsimX\Prototypes\MultiPoreWater/ReportDetail.db')
APSIMData = pd.read_sql("Select * from Report,Simulations where Report.SimulationID = Simulations.ID",con, index_col = ['TimeStep'])
APSIMData = APSIMData.drop(labels = ['SimulationID', 'ZoneName','ID'],axis=1)
PoreWater = APSIMData.ix[:,[X for X in APSIMData.columns if 'PW(' in X]]
ProfileData = pd.read_sql("Select * from Initial,Simulations where Initial.SimulationID = Simulations.ID",con)
Thickness = ProfileData.ix[:,[X for X in ProfileData.columns if 'Thick' in X]]
Timing = APSIMData.ix[:,0:3]
Timing.ix[:,'DateTime'] = [datetime.strptime(Timing.ix[X,0], '%Y-%m-%d %H:%M:%S') for X in Timing.index]
Timing.ix[:,'Date0'] = [Timing.ix[X,'DateTime'] - timedelta(hours=12) for X in Timing.index]
Timing.ix[:,'DateTime'] = [Timing.ix[X,'Date0'] + timedelta(hours=int(Timing.ix[X,'Hour'])) for X in Timing.index]
Timing.ix[:,'Now'] = [datetime.strftime(Timing.ix[X,'DateTime'],'%H:%M:%S %d-%b-%Y') for X in Timing.index]

Depth = []
depth = 0
for l in Thickness.columns:
    Depth.append(depth)
    depth += Thickness.ix[:,l][0]

LayerNum = np.size(Depth)
PoreComNum = ProfileData.ix[:,[X for X in ProfileData.columns if 'RelWC(1)' in X]].size

LayerIndices = []
for i in range(1,LayerNum +1):
    LayerIndices.append('L'+str(i))
    
CompartmentIndices = []
for i in range(1,PoreComNum +1):
    CompartmentIndices.append('C'+str(i))    

PoreColumnIndex = pd.MultiIndex.from_product([LayerIndices,CompartmentIndices],names=['Layer','PoreComponent'])
PoreWater.columns = PoreColumnIndex

PoreAxis = pd.DataFrame(data= ProfileData.ix[:,[X for X in ProfileData.columns if 'Rel' in X]].values,
                        columns = PoreColumnIndex)
PoreBounds = []
for l in range(1,LayerNum+1):
    Bounds = list(1-PoreAxis.ix[0,'L'+str(l)].values)
    Bounds.insert(0,0)
    PoreBounds.append(Bounds)
PoreX = np.array(PoreBounds)

Layers =[]
for d in Depth:
    Layers.append([d] *(PoreComNum+1))
LayersY = np.array(Layers)

PoreTicks = pd.DataFrame(data= ProfileData.ix[:,[X for X in ProfileData.columns if 'Psi' in X]].values,
                        columns = PoreColumnIndex)

Xticks = PoreTicks.ix[0,'L'+str(LayerNum)]

## Make an animated heat map of the water content in the soil

In [6]:
fig = plt.figure(figsize=(10,12))
ax1=plt.subplot2grid((7,1),(0,0),rowspan=1,xticks=[],yticks=[])
ax2=plt.subplot2grid((7,1),(2,0),rowspan=5,
                     ylabel='Soil Depth (mm)',xlabel='Large --> Small pores')
ax3=plt.subplot2grid((7,1),(1,0),rowspan=1,xticks=[],
                    ylabel = 'Pond (mm)')

ax2.invert_yaxis()
ax2.tick_params(labelsize=15)
ax2.xaxis.label.set_fontsize(30)
ax2.yaxis.label.set_fontsize(30)

pores = range(1,PoreComNum+1)
layers = Depth
Pores, Layers = np.meshgrid(pores, layers)

ims = []
for i in range(1000):
    plt1 = ax1.text(0.1,0.5,Timing.ix[i+1,'Now'],fontsize=40)
    #plt2 = ax2.pcolor(PoreX, LayersY, WetnessData(i), cmap=cm.jet_r, norm=plt.Normalize(0, 1))
    plt2 = ax2.pcolor(WetnessData(i), cmap=cm.jet_r, norm=plt.Normalize(0, 1))
    plt3 = ax3.scatter(x=range(150),y= APSIMData.ix[i+1:i+150,'Pond'])
    ims.append([plt1,plt2,plt3])
im_ani = animation.ArtistAnimation(fig, ims, interval=100, repeat_delay=1000,
                                   blit=True)
plt.show()